In [2]:
import apache_beam as beam

p = beam.Pipeline()

class AverageFn(beam.CombineFn):
  
  def create_accumulator(self):
     return (0.0, 0)   # initialize (sum, count)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    print("add_input", sum_count)
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):
    
    ind_sums, ind_counts = zip(*accumulators)       # zip - [(27, 3), (39, 3), (18, 2)]  -->   [(27,39,18), (3,3,2)]
    return sum(ind_sums), sum(ind_counts)        # (84,8)

  def extract_output(self, sum_count):    
    
    (sum, count) = sum_count    # combine globally using CombineFn
    print("extract_output", sum_count)
    return sum / count if count else float('NaN')
  

small_sum = (
           p 
            | beam.Create([15,5,7,7,9,23,13,5])
            | "Combine Globally" >> beam.CombineGlobally(AverageFn()) 
            | 'Write results' >> beam.io.WriteToText('data/combine')
          )
p.run()

add_input (0.0, 0)
add_input (15.0, 1)
add_input (20.0, 2)
add_input (27.0, 3)
add_input (34.0, 4)
add_input (43.0, 5)
add_input (66.0, 6)
add_input (79.0, 7)
extract_output (84.0, 8)
